# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [21]:
# import libraries

#Data handling
import pandas as pd
from sqlalchemy import create_engine
import re
import pickle

#NLP
import nltk
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.tokenize import word_tokenize

#Machine Learnig
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.multioutput import MultiOutputClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV

nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [2]:
#Define variables
stop_words = stopwords.words("english")
lemmatizer = WordNetLemmatizer()

In [3]:
# load data from database
engine = create_engine('sqlite:///Project2.db')
df = pd.read_sql_table('dataset_clean', engine)
#Split the train data from target varible
X = df['message']
columns = list(df.columns)
columns = columns[4:]
y = df[columns]

/opt/conda/lib/python3.6/site-packages/pandas/core/generic.py:5890: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._update_inplace(new_data)


### 2. Write a tokenization function to process your text data

In [4]:
def tokenize(text):
    # normalize case and remove punctuation
    text = re.sub(r"[^a-zA-Z0-9]", " ", text.lower())
    
    # tokenize text
    tokens = word_tokenize(text)
    
    # lemmatize andremove stop words
    tokens = [lemmatizer.lemmatize(word) for word in tokens if word not in stop_words]

    return tokens

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [5]:
#Define the initial ML pipeline with tow estimators and one transformer
pipeline = Pipeline([
        ('vect', CountVectorizer(tokenizer=tokenize)),
        ('tfidf', TfidfTransformer()),
        ('clf', MultiOutputClassifier(RandomForestClassifier()))
    ])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [6]:
#Split the data into train and test
X_train, X_test, y_train, y_test = train_test_split(X, y)
#Fit teh model
pipeline.fit(X_train, y_train)

Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [7]:
#Calculate predictions
y_pred = pipeline.predict(X_test)
#Model evaluation
for i, v in enumerate(columns):
    print('Class:',v)
    print(classification_report(y_test[v], y_pred[:,i]))
    print('\n')

Class: related
             precision    recall  f1-score   support

          0       0.60      0.45      0.52      1465
          1       0.85      0.92      0.88      5089

avg / total       0.80      0.81      0.80      6554



Class: request
             precision    recall  f1-score   support

          0       0.90      0.97      0.93      5442
          1       0.77      0.44      0.56      1112

avg / total       0.87      0.88      0.87      6554



Class: offer
             precision    recall  f1-score   support

          0       1.00      1.00      1.00      6523
          1       0.00      0.00      0.00        31

avg / total       0.99      1.00      0.99      6554



Class: aid_related
             precision    recall  f1-score   support

          0       0.74      0.84      0.79      3814
          1       0.73      0.59      0.65      2740

avg / total       0.74      0.74      0.73      6554



Class: medical_help
             precision    recall  f1-score   suppo

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


### 6. Improve your model
Use grid search to find better parameters. 

In [11]:
#Define the parameters of param grid
#Some of them are commented because the training time increases significantly
parameters = {
        'vect__ngram_range': ((1, 1), (1, 2)),
        'vect__max_df': (0.5, 1.0),
        'vect__max_features': (None, 5000),
        'tfidf__use_idf': (True, False)#,
        #'clf__n_jobs': [1, 5]#,
        #'clf__estimator': [KNeighborsClassifier(), SVC(), RandomForestClassifier()]
    }

cv = GridSearchCV(pipeline, param_grid=parameters)

cv = cv.fit(X_train, y_train)

In [12]:
#Printing the best parameters
cv.best_params_

{'tfidf__use_idf': False,
 'vect__max_df': 0.5,
 'vect__max_features': 5000,
 'vect__ngram_range': (1, 2)}

In [13]:
#Printing the best score
cv.best_score_

0.25999389685688129

In [14]:
#Making predictions with the best model
y_pred = cv.predict(X_test)

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [15]:
print("Accuracy")
print((y_pred == y_test).mean())

Accuracy
related                   0.814159
request                   0.888312
offer                     0.995270
aid_related               0.749924
medical_help              0.921117
medical_products          0.957125
search_and_rescue         0.974519
security                  0.979097
military                  0.970400
child_alone               1.000000
water                     0.960482
food                      0.944309
shelter                   0.941257
clothing                  0.986115
money                     0.979097
missing_people            0.989777
refugees                  0.964907
death                     0.964907
other_aid                 0.865273
infrastructure_related    0.936985
transport                 0.956668
buildings                 0.954989
electricity               0.978639
tools                     0.994355
hospitals                 0.988557
shops                     0.996491
aid_centers               0.988862
other_infrastructure      0.958956
weather_rel

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [17]:
#Defining a secon pipeline using a KNeighborsClassifier instead of RandomForestClassifier
pipeline2 = Pipeline([
        ('vect', CountVectorizer(tokenizer=tokenize)),
        ('tfidf', TfidfTransformer()),
        ('clf', MultiOutputClassifier(KNeighborsClassifier()))
    ])

In [18]:
#Train the second pipeline
pipeline2.fit(X_train, y_train)

Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...ric_params=None, n_jobs=1, n_neighbors=5, p=2,
           weights='uniform'),
           n_jobs=1))])

In [19]:
#Making predictions with the second pipeline
y_pred2 = pipeline2.predict(X_test)
#Evaluating the model
for i, v in enumerate(columns):
    print('Class:',v)
    print(classification_report(y_test[v], y_pred2[:,i]))
    print('\n')

Class: related
             precision    recall  f1-score   support

          0       0.64      0.23      0.34      1465
          1       0.81      0.96      0.88      5089

avg / total       0.78      0.80      0.76      6554



Class: request
             precision    recall  f1-score   support

          0       0.87      0.98      0.92      5442
          1       0.73      0.27      0.39      1112

avg / total       0.84      0.86      0.83      6554



Class: offer
             precision    recall  f1-score   support

          0       1.00      1.00      1.00      6523
          1       0.00      0.00      0.00        31

avg / total       0.99      1.00      0.99      6554



Class: aid_related
             precision    recall  f1-score   support

          0       0.63      0.94      0.76      3814
          1       0.74      0.24      0.37      2740

avg / total       0.68      0.65      0.59      6554



Class: medical_help
             precision    recall  f1-score   suppo

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


### 9. Export your model as a pickle file

In [22]:
#Save the results of gridsearchcv on a pickle file
pickle.dump(cv, open('classifier.pkl', 'wb'))

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.